## Minimal notebook

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import requests

import seaborn as sns

from collections import Counter
import itertools

from matplotlib.colors import LogNorm, SymLogNorm, Normalize
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/Users/manuelmorales/miniconda3/envs/training/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Import database

In [4]:
# always use False
LOCAL = False

if LOCAL:
    f = open('data/inspire_LA.json','r')
    df = json.load(f)
    f.close()
else:
    r = requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df = r.json()

db_master = pd.DataFrame(df)
db_master.head()

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors,gender
0,H.Albrecht.1,Venezuela,"[{'value': 'hartwig.albrecht@desy.de', 'curren...","Albrecht, Hermann","[{'value': 'H.Albrecht.1', 'schema': 'INSPIRE ...","Unlisted, VE",912061,"{'value': 'Albrecht, Hartwig', 'preferred_name...","[{'recid': 2023395, 'year': '2021', 'citation_...",[{'record': {'$ref': 'https://inspirehep.net/a...,1018731,[],male
1,D.M.B.R.1,Venezuela,None,"Bellorin, David","[{'value': 'D.M.B.R.1', 'schema': 'INSPIRE BAI'}]","Unlisted, VE",912061,"{'value': 'R., David M. Bellorin'}","[{'recid': 2023395, 'year': '2021', 'citation_...",None,2023397,[],male
2,D.F.Mundarain.1,Chile,None,"Mundarain, Douglas F.","[{'value': 'D.F.Mundarain.1', 'schema': 'INSPI...",Catolica del Norte U.,908143,"{'value': 'Mundarain, Douglas F.'}","[{'recid': 2023395, 'year': '2021', 'citation_...",None,2023398,[],male
3,R.Gaitan.2,Venezuela,"[{'value': 'rgaitan@uc.edu.ve', 'current': Fal...","Dominguez, Yessica","[{'value': 'R.Gaitan.2', 'schema': 'INSPIRE BA...","Unlisted, VE",912061,"{'value': 'Gaitan, Rolando', 'preferred_name':...","[{'recid': 1863076, 'year': '2021', 'citation_...","[{'rank': 'PHD', 'record': {'$ref': 'https://i...",1864347,[],male
4,J.Ntahompagaze.1,Rwanda,None,"Ntahompagaze, Joseph","[{'value': 'J.Ntahompagaze.1', 'schema': 'INSP...","EAIFR, Kigali",1636656,"{'value': 'Ntahompagaze, Joseph'}","[{'recid': 2087212, 'year': '2022', 'citation_...",None,2087213,[],male


# All countries in the database

In [5]:
# find all countries in json
all_countries = db_master['country'].unique()
# get rid of Nonetypes in the database
all_countries = [country for country in all_countries if country is not None]
# sort countries 
all_countries.sort()

print(len(all_countries))

for country in all_countries:
    print(country)
    
# format country 
def format_country(country):
    country = country.lower()
    ans = country.replace(" ", "_")
    return ans

115
Algeria
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bangladesh
Belarus
Belgium
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Brazil
Brunei Darussalam
Bulgaria
Canada
Chile
China
Colombia
Congo
Costa Rica
Croatia
Cuba
Cyprus
Czechia
Denmark
Dominican Republic
Ecuador
Egypt
El Salvador
Estonia
Ethiopia
Finland
France
Georgia
Germany
Ghana
Greece
Guatemala
Holy See (Vatican City State)
Honduras
Hungary
Iceland
India
Indonesia
Iran, Islamic Republic of
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Korea, Republic of
Kuwait
Kyrgyzstan
Latvia
Lebanon
Lithuania
Luxembourg
Madagascar
Malaysia
Malta
Mexico
Moldova
Mongolia
Montenegro
Morocco
Netherlands
New Zealand
Niger
Nigeria
North Macedonia
Norway
Oman
Pakistan
Palestine, State of
Panama
Paraguay
Peru
Philippines
Poland
Portugal
Puerto Rico
Qatar
Romania
Russian Federation
Rwanda
Saudi Arabia
Serbia and Montenegro
Singapore
Slovakia
Slovenia
South Africa
Spain
Sri Lanka
Sweden
Switzerland
Syrian Arab Republic
Taiwan

# Latin American countries 

In [6]:
# 19 Latin American countries indexed in the database
LA_countries = [
    'Argentina',
    'Bolivia',
    'Brazil',
    'Chile',
    'Colombia',
    'Costa Rica',
    'Cuba',
    'Dominican Republic', # new
    'Ecuador',
    'El Salvador', # new
    'Guatemala',
    'Honduras',
    'Mexico',
    'Panama', # new
    'Paraguay',
    'Peru',
    'Puerto Rico', # new
    'Uruguay',
    'Venezuela'
]

# Jamaica, Bahamas not included. 

LA_countries_format = [format_country(country) for country in LA_countries]
print(LA_countries_format)

['argentina', 'bolivia', 'brazil', 'chile', 'colombia', 'costa_rica', 'cuba', 'dominican_republic', 'ecuador', 'el_salvador', 'guatemala', 'honduras', 'mexico', 'panama', 'paraguay', 'peru', 'puerto_rico', 'uruguay', 'venezuela']


# Combined dataframes of number of papers vs year

In [7]:
# import from INSPIREHEP_new_articles_per_year.ipynb

def generate_dataframe(country: str) -> pd.DataFrame:
    """
    Generate a DataFrame whose rows are all the publications of a given country.
    The `recid` entry is unique for each publication.
    """
    # select column
    db = db_master[db_master['country']==country].reset_index(drop=True)
    
    # obtain list of the institutions of the country
    inst_list = db['institution_id'].drop_duplicates().to_list()
    
    # stack papers and select the ones from the appropriate institutions
    dp = pd.DataFrame(db.papers.apply(pd.Series).stack().to_list()).reset_index(drop=True)
    dp = dp[dp['institution_id'].isin(inst_list)]
    
    # get rid of paper duplicates
    dp = dp.drop_duplicates(subset='recid')
    
    # skip entries without listed `year`. This ignores the papers that do not have year.
    # For this reason, the sum over all years will not equal the total number of publications
    # from the other notebook. In fact, the result here will be less or equal to it. 
    dp = dp[pd.to_numeric(dp['year'], errors='coerce').notnull()].reset_index(drop=True)
    
    # transform to numerical values
    dp['year'] = pd.to_numeric(dp['year'])
    # get rid of old entries
    dp = dp[dp['year'] >= 1900]
    
    return dp


In [8]:
def generate_dataframe_latam():
    """
    """
    
    df_latam = {}
    for country in LA_countries:
        df_latam[country] = generate_dataframe(country)

    df_latam_stack = pd.concat([df_latam[country] for country in LA_countries], ignore_index=True)
    df_latam = df_latam_stack.drop_duplicates(subset='recid')

    return df_latam


In [9]:
my_df = generate_dataframe_latam()

In [10]:
my_df

,recid,year,citation_count,primary_arxiv_category,journal_id,author_id,institution_id,rank,start_date,end_date,current,institution,country
0,884148,2011,1,[hep-th],None,F.A.Schaposnik.1,902950,SENIOR,1986,None,True,La Plata U.,Argentina
1,2030209,2022,1,[hep-th],None,F.A.Schaposnik.1,902950,SENIOR,1986,None,True,La Plata U.,Argentina
2,2098035,2022,0,[],1613973,F.A.Schaposnik.1,902950,SENIOR,1986,None,True,La Plata U.,Argentina
3,1372753,2015,8,[hep-th],None,F.A.Schaposnik.1,902950,SENIOR,1986,None,True,La Plata U.,Argentina
4,693926,2005,24,[hep-th],None,F.A.Schaposnik.1,902950,SENIOR,1986,None,True,La Plata U.,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48408,249252,1987,17,[],None,A.Patino.1,905162,NaN,NaN,NaN,NaN,NaN,NaN
48409,376997,1994,0,[gr-qc],1214738,H.Rago.1,905162,NaN,NaN,NaN,NaN,NaN,NaN
48410,320615,1991,0,[],None,H.Rago.1,905162,NaN,NaN,NaN,NaN,NaN,NaN
48411,471432,1999,28,[gr-qc],1214779,H.Hernandez.2,905162,SENIOR,1999,None,True,"Andes U., Merida",Venezuela


In [11]:
def generate_articles_year_count(df):
    """
    Generate a Series whose index is the years and the values are the number of articles published in that year
    of that country

    Parameters:
        df (pd.Dataframe): The dataframe of a country or Lat. Am. generated by the function `generate_dataframe`.

    Returns:
        pd.Series: The year as index and the number of articles published in each year.
    """

    # Ensure the 'year' column is numeric and drop rows where 'year' is NaN
    df = df[pd.to_numeric(df['year'], errors='coerce').notnull()].reset_index(drop=True)
    df['year'] = pd.to_numeric(df['year'])

    # Filter out any papers before 1900 (if necessary)
    df = df[df['year'] >= 1900]

    # Create a range of years from the minimum year to 2021
    years_range = range(df['year'].min(), 2022)

    # Count the number of papers published each year
    year_counts = df['year'].value_counts().sort_index()
    year_counts = year_counts.reindex(years_range, fill_value=0)

    return year_counts


In [12]:
generate_articles_year_count(generate_dataframe('Chile'))

1959      1
1960      0
1961      0
1962      0
1963      0
       ... 
2017    294
2018    328
2019    311
2020    352
2021    401
Name: year, Length: 63, dtype: int64

In [13]:
def plot_articles_per_year_latam(save=False):
    # Generate the DataFrame for the publications of Latin America
    df = generate_dataframe_latam()

    # Generate the Series of the number of articles published each year
    year_counts = generate_articles_year_count(df)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 3))

    year_counts.plot(kind='bar', ax=ax, width=0.8)

    # Formatting the plot
    ax.set_title(f'Publications of Latin America', size=15, pad=12)
    ax.set_xlabel('Year', size=13, labelpad=8)
    ax.set_ylabel('Number', size=13)
    ax.set_axisbelow(True)
    ax.grid(True, alpha=0.4)

    # Ensure the y-axis only shows integers
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))

    # Show the plot
    plt.show()

    # Optional: Save the plot as a PDF
    if save:
        namefig = f"articles_{country}_per_year"
        fig.savefig(f'/Users/manuelmorales/inspireauth/figures/individual_articles_per_year/{namefig}.pdf', dpi=150, bbox_inches='tight')

    return 
    

In [14]:
plot_articles_per_year_latam()

# Load papers of the world

In [15]:
def load_world_papers():
    """
    Load the DataFrame of all the papers in the world.

    Returns:
        pd.DataFrame: The DataFrame of all the papers of the world. The index is the year and the column is the number of papers.
    """
    df_world = pd.read_csv('world_papers.csv')
    df_world = df_world[['year', 'Citable papers']]
    df_world = df_world.rename(columns={'Citable papers': 'World'})
    df_world = df_world.set_index('year')
    df_world = df_world[df_world.index <= 2021]

    return df_world


In [16]:
df_world = load_world_papers()
df_latam = generate_articles_year_count(generate_dataframe_latam())

In [17]:
def generate_publication_percentage(df_latam, df_world):
    """
    Generate a DataFrame with the percentage of publications of Latin America with respect to the world.

    Parameters:
        df_latam (pd.Series): The number of articles published each year in Latin America.
        df_world (pd.Series): The number of articles published each year in the world.

    Returns:
        pd.DataFrame: The percentage of publications of Latin America with respect to the world.
    """

    # Combine the two DataFrames
    df_combined = pd.concat([df_world, df_latam], axis=1)
    df_combined = df_combined.rename(columns={'Citable papers': 'World', 'year': 'LA'})

    # Calculate the percentage of publications of Latin America with respect to the world
    df_combined['Percentage'] = df_combined['LA'] / df_combined['World'] * 100

    return df_combined

In [18]:
df_combined = generate_publication_percentage(df_latam, df_world)

# Obtain the histogram

In [19]:
import matplotlib.pyplot as plt

def plot_publication_percentage(df_combined, save=False):
    # Create a figure and axis object
    fig, ax = plt.subplots(figsize=(17,4))

    # Plot the data on the axis
    ax.bar(df_combined.index, df_combined['Percentage'])

    # Set the labels and title
    ax.set_xlabel('Year', size=14, labelpad=8)
    ax.set_ylabel('Percentage', labelpad=8, size=14)
    ax.set_title(f"Latin American publications compared to global publications", size=15, pad=12)

    # Set the tick parameters
    ax.tick_params(axis='both', which='major', labelsize=12)  # Increase label size for major ticks

    ax.set_axisbelow(True)
    ax.grid(True, alpha=0.3)

    # Display the plot
    plt.show()

    # Optional: Save the plot as a PDF
    if save:
        namefig = "papers_world_comparison"
        fig.savefig(f'/Users/manuelmorales/inspireauth/figures/world_comparison/{namefig}.pdf', dpi=150, bbox_inches='tight')

In [21]:
plot_publication_percentage(df_combined, save=True)